In [62]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

#view a sample of the data
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [2]:
#Calculate the summary information required
unique_school_names = school_data_complete["school_name"].unique()
total_schools = len(unique_school_names)
total_students = school_data_complete["Student ID"].count()
unique_budget_figures = school_data_complete["budget"].unique()
total_budget = unique_budget_figures.sum()
avg_math_score = school_data_complete["maths_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()
per_pass_math = len(school_data_complete.loc[school_data_complete["maths_score"] >= 50, :])/total_students*100
per_pass_read = len(school_data_complete.loc[school_data_complete["reading_score"] >= 50, :])/total_students*100
num_pass_overall = len(school_data_complete.loc[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50), :])
per_pass_overall = num_pass_overall/total_students*100

#Create data frame with the results
LGA_Summary = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":[total_students],
    "Total Budget":[total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score":[avg_read_score],
    "% Passing Maths":[per_pass_math],
    "% Passing Reading":[per_pass_read],
    "% Overall Passing":[per_pass_overall]
})
#Clean up the format of the results
LGA_Summary["Total Students"] = LGA_Summary["Total Students"].map('{:,.0f}'.format)
LGA_Summary["Total Budget"] = LGA_Summary["Total Budget"].map('${:,.0f}'.format)
LGA_Summary["Average Math Score"] = LGA_Summary["Average Math Score"].map('{:,.2f}%'.format)
LGA_Summary["Average Reading Score"] = LGA_Summary["Average Reading Score"].map('{:,.2f}%'.format)
LGA_Summary["% Passing Maths"] = LGA_Summary["% Passing Maths"].map('{:,.2f}%'.format)
LGA_Summary["% Passing Reading"] = LGA_Summary["% Passing Reading"].map('{:,.2f}%'.format)
LGA_Summary["% Overall Passing"] = LGA_Summary["% Overall Passing"].map('{:,.2f}%'.format)

LGA_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",70.34%,69.98%,86.08%,84.43%,72.81%


In [38]:
#Take columns from 

school_summary

,School Name,School Type,Total Students,Total School Budget
0,Huang High School,Government,2917,1910635
1,Figueroa High School,Government,2949,1884411
2,Shelton High School,Independent,1761,1056600
3,Hernandez High School,Government,4635,3022020
4,Griffin High School,Independent,1468,917500
5,Wilson High School,Independent,2283,1319574
6,Cabrera High School,Independent,1858,1081356
7,Bailey High School,Government,4976,3124928
8,Holden High School,Independent,427,248087
9,Pena High School,Independent,962,585858


In [76]:
#Take columns from 
school_summary = school_data[["school_name","type","size","budget"]]
school_summary = school_summary.rename(columns={"school_name":"School Name",
                     "type":"School Type",
                     "size":"Total Students",
                     "budget":"Total School Budget"})
school_summary.set_index("School Name", inplace=True)
school_summary.sort_index(ascending=True, inplace=True)

#Calculate statistics for each school
#@TODO fix the Overall Passing Calculations
school_summary["Average Maths Score"] = school_data_complete.groupby(["school_name"])["maths_score"].mean()
school_summary["Average Reading Score"] = school_data_complete.groupby(["school_name"])["reading_score"].mean()
school_summary["Per Student Budget"]=school_summary["Total School Budget"]/school_summary["Total Students"]
school_summary["% Passing Math"] = school_data_complete.groupby(["school_name"]).apply(lambda x:x[x["maths_score"] >= 50]["maths_score"].count())/school_summary["Total Students"]*100
school_summary["% Passing Reading"] = school_data_complete.groupby(["school_name"]).apply(lambda x:x[x["reading_score"] >= 50]["reading_score"].count())/school_summary["Total Students"]*100
school_summary["% Overall Passing"] = school_summary["% Passing Math"]*school_summary["% Passing Reading"]/100

#Reorganise the headers
school_summary=school_summary[["School Type","Total Students","Total School Budget","Per Student Budget",
                               "Average Maths Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]]
#Clean up the format of the results
school_summary["Total Students"] = school_summary["Total Students"].map('{:,.0f}'.format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map('${:,.0f}'.format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map('${:,.0f}'.format)
school_summary["Average Maths Score"] = school_summary["Average Maths Score"].map('{:,.2f}%'.format)
school_summary["Average Reading Score"] = school_summary["Average Reading Score"].map('{:,.2f}%'.format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map('{:,.2f}%'.format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map('{:,.2f}%'.format)
school_summary["% Overall Passing"] = school_summary["% Overall Passing"].map('{:,.2f}%'.format)

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,Government,"4,976","$3,124,928",$628,72.35%,71.01%,91.64%,87.38%,80.07%
Cabrera High School,Independent,"1,858","$1,081,356",$582,71.66%,71.36%,90.85%,89.07%,80.92%
Figueroa High School,Government,"2,949","$1,884,411",$639,68.70%,69.08%,81.65%,82.81%,67.62%
Ford High School,Government,"2,739","$1,763,916",$644,69.09%,69.57%,82.44%,82.22%,67.78%
Griffin High School,Independent,"1,468","$917,500",$625,71.79%,71.25%,91.21%,88.49%,80.71%
Hernandez High School,Government,"4,635","$3,022,020",$652,68.87%,69.19%,80.95%,81.88%,66.28%
Holden High School,Independent,427,"$248,087",$581,72.58%,71.66%,89.93%,88.52%,79.61%
Huang High School,Government,"2,917","$1,910,635",$655,68.94%,68.91%,81.69%,81.45%,66.54%
Johnson High School,Government,"4,761","$3,094,650",$650,68.84%,69.04%,82.06%,81.98%,67.27%


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [ ]:
top_5_schools = school_summary.loc()

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Year

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type